<a href="https://colab.research.google.com/github/Bigizic/get_real/blob/main/coding_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pathlib dotenv openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=acc7aa08bc29f0135eeb5aba77bfc3c0daf996c483aeeb7a2c7c09276389494f
  Stored in directory: /root/.cache/pip/wheels/9d/67/2f/53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built typing


In [ ]:
#!/usr/bin/env python3
from pathlib import Path
from typing import List

def smart_sample_js_files(base_path: Path, inputs: List[str]):
    print("🔍 Smart-sampling JS files...")

    input_paths = [(base_path / i).resolve() for i in inputs]
    js_files = []

    for path in input_paths:
        if not path.exists():
            print(f"⚠️ Path does not exist: {path}")
            continue

        if path.is_file() and path.suffix in [".js", ".jsx"]:
            js_files.append(path)
        elif path.is_dir():
            for f in path.rglob("*"):
                if (
                    f.suffix in [".js", ".jsx"]
                    and all(skip not in f.parts for skip in ["node_modules", "build", "dist", "Gpt"])
                ):
                    js_files.append(f)

    summaries = []
    for file_path in js_files:
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()

            total_lines = len(lines)
            if total_lines <= 10000:
                sample = lines
            else:
                sample = (
                    lines[:40]
                    + ["\n// ...\n"]
                    + lines[total_lines // 2 - 15: total_lines // 2 + 15]
                    + ["\n// ...\n"]
                    + lines[-30:]
                )

            summary = f"\n\n// === FILE: {file_path.relative_to(base_path)} ===\n"
            summary += "".join(sample)
            summaries.append(summary)
        except Exception as e:
            print(f"⚠️ Skipping file {file_path}: {e}")

    print(f"✅ {len(summaries)} files summarized.")
    return "\n".join(summaries)


In [ ]:
#!/usr/bin/env python3
import os
import cmd
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI

from summarizer import smart_sample_js_files  # 🔥 Import your summarizer

# Load .env
load_dotenv()

client = OpenAI()  # Will use the OPENAI_API_KEY from environment variable

BASE_DIR = Path.cwd()
GPT_DIR = BASE_DIR / "Gpt"
SUMMARY_CONTEXT = ""


class GptCodeEditor(cmd.Cmd):
    intro = "🤖 GPT Code Editor started. Type 'help' for options.\n"
    prompt = "(gpt) "
    use_rawinput = True

    def preloop(self):
        self.summarized = False
        print("Do you want to summarize specific parts of your JS codebase to help GPT understand?")
        choice = input("1. Yes\n2. No\nEnter 1 or 2: ").strip()
        if choice == "1":
            targets = input(
                "Enter relative paths to directories or files to summarize (comma-separated):\n"
                "e.g. client/components, server/models/user.js\n> "
            ).strip()
            target_list = [t.strip() for t in targets.split(",") if t.strip()]
            if target_list:
                self.summarize_codebase(target_list)
                print("✅ Summary sent to OpenAI. GPT will now have better context.")
                self.summarized = True
            else:
                print("⚠️ No valid paths provided. Skipping summarization.")
        else:
            print("🔎 Skipping codebase summary.")

        self.ask_main_action()


    def summarize_codebase(self, inputs):
        global SUMMARY_CONTEXT
        SUMMARY_CONTEXT = ""
        SUMMARY_CONTEXT = smart_sample_js_files(BASE_DIR, inputs)

    def ask_main_action(self):
        print("What do you want to do?")
        print("1. Create a new file")
        print("2. Update an existing file")
        print("3. Suggest full code changes based on my prompt")
        choice = input("Enter 1, 2 or 3: ").strip()

        if choice == "1":
            file_name = input("Enter new file path (relative, e.g. server/models/newsletter.js): ").strip()
            prompt = input("Describe what the new file should contain:\n")
            self.create_file(file_name, prompt)
        elif choice == "2":
            file_path = input("Enter existing file path (relative): ").strip()
            self.do_update(file_path)
        elif choice == "3":
            prompt = input("Describe the feature or update you want across your project:\n")
            self.do_project_wide_suggestion(prompt)
        else:
            print("❌ Invalid choice.")
            self.ask_main_action()

    def do_update(self, arg):
        "Update an existing file: update <relative_path_to_file>"
        file_path = (BASE_DIR / arg).resolve()
        if not file_path.exists():
            print(f"❌ File not found: {file_path}")
            return

        print(f"✅ File found: {file_path}")
        prompt = input("Enter your prompt: ")
        print("🛠️  Working on file...")

        with open(file_path, "r", encoding="utf-8") as f:
            original_code = f.read()

        gpt_response = self.send_to_gpt(original_code, prompt)

        output_file = GPT_DIR / Path(arg)
        output_file.parent.mkdir(parents=True, exist_ok=True)
        with open(output_file, "w", encoding="utf-8") as f:
            f.write("// === ORIGINAL CODE ===\n")
            f.write(original_code)
            f.write("\n\n// === GPT EDITED CODE ===\n")
            f.write(gpt_response)

        print(f"✅ Saved updated code to: {output_file}")
        self.ask_main_action()

    def create_file(self, file_path, prompt):
        print("🛠️  Creating new file...")

        gpt_response = self.send_to_gpt("", prompt)

        output_file = GPT_DIR / file_path
        output_file.parent.mkdir(parents=True, exist_ok=True)
        with open(output_file, "w", encoding="utf-8") as f:
            f.write("// === GPT GENERATED CODE ===\n")
            f.write(gpt_response)

        print(f"✅ New file created at: {output_file}")
        self.ask_main_action()

    def send_to_gpt(self, code, user_prompt):
        messages = [
            {
                "role": "system",
                "content": "You are a programming assistant. Generate or modify JavaScript code as instructed."
            }
        ]
        if self.summarized and SUMMARY_CONTEXT:
            messages.append({
                "role": "user",
                "content": f"My codebase contains these files:\n{SUMMARY_CONTEXT[:12000]}..."
            })

        messages.append({
            "role": "user",
            "content": (
                f"Here is the file code:\n{code}\n\nInstruction:\n{user_prompt}"
                if code else f"Instruction:\n{user_prompt}"
            )
        })

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.3
        )

        return response.choices[0].message.content

    def do_project_wide_suggestion(self, prompt):
        if not self.summarized or not SUMMARY_CONTEXT:
            print("⚠️ You must summarize the codebase first.")
            return

        print("🤖 Working on full project suggestion...")

        messages = [
            {
                "role": "system",
                "content": (
                    "You are a senior full-stack developer helping another senior full-stack developer. "
                    "The project is a full-stack JavaScript web app using React (with Redux for state management) on the frontend, "
                    "and Express.js with MongoDB on the backend. "
                    "You're to suggest specific and practical code updates across multiple files as needed. "
                    "For each suggestion, provide: the file path, a short description of the change, and the actual code blocks if relevant. "
                    "Changes should reflect realistic code architecture and match existing patterns."
                    "you can create new components to make code architecture better, you're to read the whole file and suggest best practices to go alongside what's currently been shown to you"
                )
            },
            {
                "role": "user",
                "content": f"I'm working on a full-stack web application. My project includes these files:\n{SUMMARY_CONTEXT[:12000]}..."
            },
            {
                "role": "user",
                "content": f"Based on the above structure, suggest the full code changes required to implement this feature:\n\n{prompt}"
            }
        ]

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.3
        )

        full_suggestion = response.choices[0].message.content

        output_file = GPT_DIR / "changes" / "project_wide_suggestion.js"
        output_file.parent.mkdir(parents=True, exist_ok=True)
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(full_suggestion)

        print(f"✅ Suggestion saved to: {output_file}")
        self.ask_main_action()

    def do_exit(self, arg):
        "Exit the assistant"
        print("👋 Goodbye!")
        return True

    def do_help(self, arg):
        print("""
Available Commands:
  update <path>   - Update an existing file using GPT
  exit            - Quit the tool
  help            - Show this help message
  (other actions are prompted interactively on start)
""")


def start():
    cli = GptCodeEditor()
    cli.cmdloop()


if __name__ == "__main__":
    start()